# NewCo Hourly Urban Emissions Data - Dataset Cleaning
@mattyarri

**Date:** Feb 2, 2022

**Goal:** We now have hourly urban emissions data, by census block, across a 8 broad sectors of the economy. We want to create some exploratory views of this data, to support an upcoming meeting between the NewCo team and Jane Lubchenko on Feburary 10th. Some things we want to be able to do in this notebook:
* Query the hourly dataset, create new columns showing aggregations for hourly, weekly, and annual emissions
* Merge the dataset, which is at the census block level, with data from the ACS, to enable per-capita emissions
* Show aggregations at the urban area and state level
* Plot data on a map across the US
* Prototype widgets (ie dropdown menus) allowing user to quickly toggle between states, urban areas, etc
* Write function to show "similar cities" based on emission profiles. This includes things like total aggregate emissions for a given sector, but also things like emissions composition (e.g. 30% come from residential)

**Notebook Comment:** Due to the large file sizes involved, this notebook purely handles dataset cleaning and aggregation. A separate notebook file handles all of the visualization schemes.

In [ ]:
import sqlalchemy
import pandas as pd
import datetime
import geopandas as gpd
import plotly.express as px
import plotly.io as pio
import numpy as np
import gc
import math

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pio.templates.default = "none"
%config InlineBackend.figure_format ='retina'

# engine = sqlalchemy.create_engine(
#     sqlalchemy.engine.url.URL(
#         drivername="postgresql",
#         username="postgres",
#         password="UOYo0U7Bz7hl79oABhLP",
#         host="vulcan.c1sk5ntzwqee.us-east-1.rds.amazonaws.com",
#         port="5432",
#         database="vulcan",
#     ),
#     echo_pool=True,
# )
# print("connecting with engine " + str(engine))
# connection = engine.connect()

In [ ]:
# First let's read in raw hourly emissions dataset
vulcan_df = pd.read_csv('raw_data_not_on_github/total_2015_hourly.csv')
vulcan_df.head()
vulcan_df.shape

## Dataset Explore (Without Population Data)
First let's start making some dashboard views using just the raw data provided by the Vulcan team, without trying to merge yet any population or economic data.

In [ ]:
# Change column naming to drop H's from dataset
def rename_hvars(vname):
    try:
        hstring, hour = vname.split('h')
        return int(hour)
    except:
        return vname
vulcan_df = vulcan_df.rename(rename_hvars,axis='columns') #Note: Will break if any other column has "h" in its name
vulcan_df.head()

In [ ]:
#Let's do a quick check for null counts here by row. We'll just sample two columns out of the hourly values for now
vulcan_df[['GEOID10','Sector',1,8760]].isna().sum()

vulcan_df.shape

printmd('#### So an important note here: About 45% of row entries here are null values. Lets do a little more digging here, but definitely a question for Geoff.')

In [ ]:
#Let's just check if null values for sensible sectors (airports, marine vehicles, etc)
t_df = vulcan_df[['Sector',1]]
t_df['null_bool'] = t_df[1].isna()
t_df['row_count'] = 1
t_df = t_df.groupby('Sector').sum().reset_index()
t_df['pct_null'] = t_df['null_bool'] / t_df['row_count']
t_df

printmd('#### A lot of these sectors have high percentages of null values. Marine vehicles makes sense, \
        but cement, nonroad, and electricity production seem very suspect.')

In [ ]:
#Ok, the next thing we want to do is create 5 dataframes: an hourly, daily, weekly monthly, and yearly dataframe for emissions
#Note: There's a more elegant way to do this, but the elegant way broke memory constraints unfortunately
#Note: It is very important that we do date logic here BEFORE we try to do any dataframe melts. Otherwise we explode memory constraints
vulcan_df.shape

def create_date(h):
    start = datetime.datetime(2015, 1, 1)
    return start + datetime.timedelta(hours=h-1) #Rezero hourly integer column

print('Converting hourly column headers to dates...')
t_df = vulcan_df.rename(lambda x: create_date(x) if isinstance(x,int) else x,axis='columns')

print('Melting dataframe to long format...')
long_df = t_df.melt(id_vars = ['GEOID10','Sector'],
                    var_name = 'Hour',
                    value_name = 'tC')

print('Converting hours to days...')
long_df['Day'] = long_df.Hour.dt.to_period('D')
print('Converting hours to weeks...')
long_df['Week'] = long_df.Hour.dt.to_period('W').apply(lambda r: r.start_time)
print('Converting hours to months...')
long_df['Month'] = long_df.Hour.dt.to_period('M')
print('Converting hours to years...')
long_df['Year'] = long_df.Hour.dt.to_period('Y')

print('Reordering columns...')
#Re-order columsn for cosmetics, write to output file just to save progress
long_df = long_df[['GEOID10','Sector','Hour','Day','Week','Month','Year','tC']]
print('Writing dataset to disk...')
long_df.to_pickle('output_data/vulcan_unaggregated_datetime_dataset.pkl')
long_df.sample(5)
long_df.shape

In [ ]:
# This was pretty memory intensive, so we're doing to do a little housekeeping here
try:
    del long_df
except: print('No long_df')
try:
    del t_df
except: print('No t_df')
try:
    del vulcan_df
except: print('No vulcan_df')
gc.collect()
long_df = pd.read_pickle('output_data/vulcan_unaggregated_datetime_dataset.pkl')
long_df.dtypes
long_df.head()

In [ ]:
#Next we want to get some aggregations together of all of these. Again, want to see how quickly these go
print('Creating Hourly Dataframe')
long_df_hour = long_df[['GEOID10','Sector','Hour','tC']]
print('Creating Daily Dataframe')
long_df_day = long_df[['GEOID10','Sector','Day','tC']].groupby(['GEOID10','Sector','Day']).sum().reset_index()
print('Creating Weekly Dataframe')
long_df_week = long_df[['GEOID10','Sector','Week','tC']].groupby(['GEOID10','Sector','Week']).sum().reset_index()
print('Creating Monthly Dataframe')
long_df_month = long_df[['GEOID10','Sector','Month','tC']].groupby(['GEOID10','Sector','Month']).sum().reset_index()
print('Creating Yearly Dataframe')
long_df_year = long_df[['GEOID10','Sector','Year','tC']].groupby(['GEOID10','Sector','Year']).sum().reset_index()

long_df_hour.head()
long_df_day.head()
long_df_week.head()
long_df_month.head()
long_df_year.head()

In [ ]:
# Write these new datasets to hard disk
long_df_hour.to_pickle('output_data/hour_tc_dataset.pkl')
long_df_day.to_pickle('output_data/day_tc_dataset.pkl')
long_df_week.to_pickle('output_data/week_tc_dataset.pkl')
long_df_month.to_pickle('output_data/month_tc_dataset.pkl')
long_df_year.to_pickle('output_data/year_tc_dataset.pkl')

In [ ]:
long_df_hour = pd.read_pickle('output_data/hour_tc_dataset.pkl')
long_df_day = pd.read_pickle('output_data/day_tc_dataset.pkl')
long_df_week = pd.read_pickle('output_data/week_tc_dataset.pkl')
long_df_month = pd.read_pickle('output_data/month_tc_dataset.pkl')
long_df_year = pd.read_pickle('output_data/year_tc_dataset.pkl')

In [ ]:
#Read in shapefile dataset
shape_df = gpd.read_file('raw_data_not_on_github/cb_2018_us_ua10_500k/cb_2018_us_ua10_500k.shp')
shape_df.head()
shape_df.shape

In [ ]:
# Convert GEOID10 to integer within shapefile dataset to handle leading zeros
shape_df['GEOID10_int'] = shape_df.GEOID10.apply(int)

hour_df = long_df_hour.merge(shape_df[['GEOID10_int','NAME10','ALAND10','AWATER10','geometry']],
                           how = 'left',
                           left_on = 'GEOID10',
                           right_on = 'GEOID10_int'
                           )
hour_df.head()

day_df = long_df_day.merge(shape_df[['GEOID10_int','NAME10','ALAND10','AWATER10','geometry']],
                           how = 'left',
                           left_on = 'GEOID10',
                           right_on = 'GEOID10_int'
                           )
day_df.head()

week_df = long_df_week.merge(shape_df[['GEOID10_int','NAME10','ALAND10','AWATER10','geometry']],
                           how = 'left',
                           left_on = 'GEOID10',
                           right_on = 'GEOID10_int'
                           )
week_df.head()

month_df = long_df_month.merge(shape_df[['GEOID10_int','NAME10','ALAND10','AWATER10','geometry']],
                           how = 'left',
                           left_on = 'GEOID10',
                           right_on = 'GEOID10_int'
                           )
month_df.head()

year_df = long_df_year.merge(shape_df[['GEOID10_int','NAME10','ALAND10','AWATER10','geometry']],
                           how = 'left',
                           left_on = 'GEOID10',
                           right_on = 'GEOID10_int'
                           )
year_df.head()

In [ ]:
#Next we want to just clean these up a little bit in both ordering and column naming

def quick_df_cleanup(df,period_id):
    df = df.rename(columns = {'NAME10':'Urban Area','ALAND10':'Land Area','AWATER10':'Water Area'})
    df = df[['GEOID10','Sector','Urban Area','Land Area','Water Area','geometry',period_id,'tC']]
    return df

hour_geo = quick_df_cleanup(hour_df,'Hour')
day_geo = quick_df_cleanup(day_df,'Day')
week_geo = quick_df_cleanup(week_df,'Week')
month_geo = quick_df_cleanup(month_df,'Month')
year_geo = quick_df_cleanup(year_df,'Year')

In [ ]:
#Write these to disk as well
hour_geo.to_pickle('output_data/hour_geo.pkl')
day_geo.to_pickle('output_data/day_geo.pkl')
week_geo.to_pickle('output_data/week_geo.pkl')
month_geo.to_pickle('output_data/month_geo.pkl')
year_geo.to_pickle('output_data/year_geo.pkl')

### Reduced Shapefile Merge with Better Memory Management

In [ ]:
# long_df_hour = pd.read_pickle('output_data/hour_tc_dataset.pkl')
# long_df_day = pd.read_pickle('output_data/day_tc_dataset.pkl')
# long_df_week = pd.read_pickle('output_data/week_tc_dataset.pkl')
# long_df_month = pd.read_pickle('output_data/month_tc_dataset.pkl')
long_df_year = pd.read_pickle('output_data/year_tc_dataset.pkl')

In [ ]:
#Read in shapefile dataset
shape_df = gpd.read_file('raw_data_not_on_github/cb_2018_us_ua10_500k/cb_2018_us_ua10_500k.shp')
shape_df.head()
shape_df.shape

In [ ]:
# Convert GEOID10 to integer within shapefile dataset to handle leading zeros
shape_df['GEOID10_int'] = shape_df.GEOID10.apply(int)

period_list = ['Year','Month','Week','Day','Hour']

def quick_df_cleanup(df,period_id):
    df = df.rename(columns = {'NAME10':'Urban Area','ALAND10':'Land Area','AWATER10':'Water Area'})
    df = df[['GEOID10','Sector','Urban Area','Land Area','Water Area','geometry',period_id,'tC']]
    return df

for p in period_list:
    print('Starting '+ p)
    p_lower = p.lower()
    long_df = pd.read_pickle('output_data/'+p_lower+'_tc_dataset.pkl')
    t_df = long_df.merge(shape_df[['GEOID10_int','NAME10','ALAND10','AWATER10','geometry']],
                           how = 'left',
                           left_on = 'GEOID10',
                           right_on = 'GEOID10_int'
                           )
    t_df.head()
    quick_df_cleanup(t_df,p).to_pickle('output_data/'+p_lower+'_geo.pkl')
    print('Finished writing file for '+p)
    del long_df
    del t_df
    gc.collect()
    print('Finished loop for '+p)